In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import utils

In [ ]:
model = ResNet50(include_top = True, weights = 'imagenet')
model.summary()
model.save('model.h5')

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [ ]:
model_pretrained = tf.keras.models.load_model('model.h5')

model_cifar10 = model_pretrained.layers[-2].output
fc_layer = Dense(10, activation = "softmax")(model_cifar10)
model_cifar10 = tf.keras.models.Model(inputs = model_pretrained.input, outputs = fc_layer)

model_config = model_cifar10.get_config()
model_config['layers'][0]['config']['batch_input_shape'] = (None, 32, 32, 3)
model_cifar10 = model.__class__.from_config(model_config, custom_objects={})

model_cifar10.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [ ]:
model_cifar10.compile(optimizer='adam',loss='categorical_crossentropy',
                      metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
model_cifar10.fit(x_train, y_train, epochs=10, batch_size=2000, validation_data=(x_test, y_test), callbacks=[callback])

Epoch 1/10
25/25 [==============================] - 3191s 126s/step - loss: 2.0930 - accuracy: 0.3279 - val_loss: 50.8024 - val_accuracy: 0.0996
Epoch 2/10
25/25 [==============================] - 3144s 126s/step - loss: 1.3700 - accuracy: 0.5000 - val_loss: 2.3100 - val_accuracy: 0.1679
Epoch 3/10
25/25 [==============================] - 3167s 127s/step - loss: 1.1304 - accuracy: 0.5896 - val_loss: 2.2088 - val_accuracy: 0.1708
Epoch 4/10
25/25 [==============================] - 3187s 128s/step - loss: 0.9489 - accuracy: 0.6611 - val_loss: 2.1247 - val_accuracy: 0.2051
Epoch 5/10
25/25 [==============================] - 3057s 122s/step - loss: 0.8172 - accuracy: 0.7104 - val_loss: 1.9339 - val_accuracy: 0.2897
Epoch 6/10
25/25 [==============================] - 3026s 121s/step - loss: 0.6760 - accuracy: 0.7618 - val_loss: 1.7357 - val_accuracy: 0.4011
Epoch 7/10
 9/25 [=========>....................] - ETA: 32:13 - loss: 0.5393 - accuracy: 0.8096